In [ ]:
# !pip install patool
!pip install mediapipe


In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout, GRU, Conv2D, Conv1D, Flatten, Bidirectional, SimpleRNN
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import mediapipe as mp
# import pickle as pk
from google.colab import drive
from google.colab import files

In [ ]:
drive.mount("/content/drive", force_remount=True)
%cd drive/MyDrive/Colab Notebooks/

In [ ]:
# !pip install patool
# import patoolib
# patoolib.extract_archive("./traindata/all_cut.rar", outdir="./traindata")

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    refine_face_landmarks=False,
    min_detection_confidence=0.2,
    min_tracking_confidence=0.2)

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    return  results

In [ ]:
def normalize_zscore(pose, face, lh, rh):
       

    pose = (pose - pose.mean(axis=0))/(pose.std(axis=0) + 1e-7)
    face = (face -  face.mean(axis=0))/(face.std(axis=0) + 1e-7)
    lh = (lh - lh.mean(axis=0))/(lh.std(axis=0) + 1e-7)
    rh = (rh - rh.mean(axis=0))/(rh.std(axis=0) + 1e-7)

    return pose, face, lh, rh

In [ ]:
def extract_keypoints(results):

    # pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((33,3))
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros((468,3))
    # lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,3))
    # rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21,3))
    pose = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((33,2))
    face = np.array([[res.x, res.y] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros((468,2))
    lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,2))
    rh = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21,2))

    pose, face, lh, rh = normalize_zscore(pose, face, lh, rh)
    face = face[:10,:]
    
    return np.concatenate([face.flatten(), pose.flatten(), lh.flatten(), rh.flatten()])
      

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def videoProc2(path, c=1, skip=0, o=False):
    "###"


    cap = cv2.VideoCapture(path)
    success = True
    
    framecount = 20

    fnum = 0
    v = []
    
    fpsCounter = 0


    while success:     
        success, frame = cap.read()
        
        if fpsCounter<=0:
            fpsCounter=c
        else:
            fpsCounter-=1           
            continue
        if fnum == framecount:
          break 
        
        if success:  
            # frame = cv2.resize(frame, (720,480), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)

            eres = extract_keypoints(results)
            tmp = np.reshape(eres, (1,-1))
            v.append(tmp)

            fnum += 1
            if o:
                draw_styled_landmarks(frame, results)
                plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                plt.show()


    
    v = np.reshape(v,(fnum,-1)) if fnum >0 else[]
        
    if fnum<framecount and fnum>0:
        tmp = np.zeros((framecount-fnum,170))
        v = np.concatenate((v, tmp), axis=0) 


    return v

***
***
***

In [ ]:
path ="./traindata/all"
file_list = os.listdir(path)
files = file_list
files.sort()

In [ ]:
len(files)

In [ ]:
files = [path+"/"+f for f in files]# 
files[0],files[499]

In [ ]:
v= videoProc2(files[50],c=5,o=1)
v.shape

In [ ]:
v[14].max(),v[14].min(),

In [ ]:
v[14].std(),v[14].mean()

In [ ]:
sum = 0
for i in range(0,3200,25):
  v= videoProc2(files[i],c=5,o=0)
  sum+=v.shape[0]
  print(v.shape,int(i/50))


words={0:'Opaque',1:'Red',2:'Green',3:'Yellow',4:'Bright',5:'Light-blue	',6:'Colors',7:'Pink',8:'Women',9:'Enemy',10:'Son',11:'Man',12:'Away',13:'Drawer',14:'Born',15:'Learn',
       16:'Call',17:'Skimmer',18:'Bitter',19:'Sweet milk',20:'Milk',21:'Water',22:'Food',23:'Argentina',24:'Uruguay',25:'Country',26:'Last name',27:'Where',28:'Mock',
       29:'Birthday',30:'Breakfast',31:'Photo',32:'Hungry',33:'Map',34:'Coin',35:'Music',36:'Ship',37:'None',38:'Name',39:'Patience',40:'Perfume',41:'Deaf',42:'Trap',
       43:'Rice',44:'Barbecue',45:'Candy',46:'Chewing-gum',47:'Spaghetti',48:'Yogurt',49:'Accept',50:'Thanks',51:'Shut down',52:'Appear',53:'To land',54:'Catch',
       55:'Help',56:'Dance',57:'Bathe',58:'Buy',59:'Copy',60:'Run',61:'Realize',62:'Give',63:'Find'}

In [ ]:
words2={0:'Opaque',1:'Red',2:'Green',3:'Yellow',4:'Bright',5:'Light-blue',6:'Colors',7:'Pink',8:'Women',9:'Enemy',10:'Son',11:'Man',12:'Away',13:'Drawer',14:'Born',15:'Learn',
       16:'Call',17:'Skimmer',18:'Bitter',19:'Sweet milk',20:'Milk',21:'Water',22:'Food',23:'Argentina',24:'Uruguay',25:'Country',26:'Last name',27:'Where',
       29:'Birthday',32:'Hungry',36:'Ship',37:'None',38:'Name',39:'Patience',40:'Perfume',41:'Deaf',45:'Candy',46:'Chewing-gum'
       ,51:'Shut down',58:'Buy',61:'Realize',63:'Find'}

In [ ]:
words2l={}

In [ ]:
k=0
for i,w in words2.items():
  words2l[w]=k
  k+=1

In [ ]:
words2l

In [ ]:
l=0
X_train=[]
X_dev=[]
y_train=[]
y_dev=[]
files_ = files[1600:3200]
for f in files_:
    v= videoProc2(f,c=5,o=0)


    p = int(f.split('/')[-1].split('_')[1])
    if p <10:
      X_train.append(v)
      label = int(f.split('/')[-1].split('_')[0])-1
      y_train.append(label)
    else:
      X_dev.append(v)
      label = int(f.split('/')[-1].split('_')[0])-1
      y_dev.append(label)


    print(p, l)
    l+=1

    

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_dev = np.array(X_dev)
y_dev = np.array(y_dev)

In [ ]:
X_train.shape,y_train.shape,X_dev.shape,y_dev.shape

In [ ]:
np.save("./X_train_5.npy",X_train)
np.save("./y_train_5.npy",y_train)
np.save("./X_dev_5.npy",X_dev)
np.save("./y_dev_5.npy",y_dev)

In [ ]:
del files
del file_list

***
***
***

In [ ]:
# X_train = np.load("./traindata/X_train_1.npy")
# y_train = np.load("./traindata/y_train_1.npy")
# X_dev = np.load("./traindata/X_dev_1.npy")
# y_dev = np.load("./traindata/y_dev_1.npy")
X_train = np.load("./X_train_4.npy")
y_train = np.load("./y_train_4.npy")
X_dev = np.load("./X_dev_4.npy")
y_dev = np.load("./y_dev_4.npy")

In [ ]:
# X_train2 = np.load("./traindata/X_train_2.npy")
# y_train2 = np.load("./traindata/y_train_2.npy")
# X_dev2 = np.load("./traindata/X_dev_2.npy")
# y_dev2 = np.load("./traindata/y_dev_2.npy")
X_train2 = np.load("./X_train_5.npy")
y_train2 = np.load("./y_train_5.npy")
X_dev2 = np.load("./X_dev_5.npy")
y_dev2 = np.load("./y_dev_5.npy")

In [ ]:
X_train = np.concatenate([X_train,X_train2],axis=0)
y_train = np.concatenate([y_train,y_train2],axis=0)
X_dev = np.concatenate([X_dev,X_dev2],axis=0)
y_dev = np.concatenate([y_dev,y_dev2],axis=0)


In [ ]:
del X_train2
del y_train2
del X_dev2
del y_dev2

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=13)

X_dev, y_dev = shuffle(X_dev,  y_dev, random_state=13)

In [ ]:
m=X_train.mean()
s=(X_train.std()+1e-7)

In [ ]:
m,s

In [ ]:
# X_train = ((X_train - m)/ s)
# X_dev = (X_dev-m)/s

In [ ]:
# X_train = X_train[:,:,np.newaxis]
# X_dev = X_dev[:,:,np.newaxis]

In [ ]:
X_train[13].mean(),X_train[13].std(),X_train.shape

In [ ]:
X_train[3].min(),X_train[3].max(),X_train[3].shape

In [ ]:
y_train[0:10],y_train.min(),y_train.max(),y_train.shape

In [ ]:


# Bidirectional(GRU(128, return_sequences=True, activation='tanh')),
# BatchNormalization(axis=-1,center=True,scale=True,),
# ReLU(), 
# LSTM
# tf.keras.layers.SimpleRNN(256, return_sequences=True, activation="relu",),


# Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,0.05)),  
# # Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,0.05)),     recurrent_regularizer=tf.keras.regularizers.L1L2(1e-4,1e-2)
# BatchNormalization(axis=-1,center=True,scale=True,), recurrent_regularizer=tf.keras.regularizers.L1L2(1e-4,1e-4),


model_lstm_1 = Sequential([
tf.keras.Input(shape=(20,170),),


GRU(64, return_sequences=False, activation="relu",),
BatchNormalization(axis=-1,center=True,scale=True,),
Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,0.01)),
BatchNormalization(axis=-1,center=True,scale=True,),
Dropout(0.4),


Dense(64, activation='softmax')
])


In [ ]:

model_lstm_1.summary()

In [ ]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, patience=20,mode="min")
callback2 = tf.keras.callbacks.EarlyStopping(monitor="val_sparse_categorical_accuracy", restore_best_weights=True,mode="max",patience=20)



In [ ]:
model_lstm_1.compile(tf.keras.optimizers.Adam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999,epsilon=1e-07,)
        ,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["sparse_categorical_accuracy"])

In [ ]:
model_lstm_1.fit(X_train, y_train, epochs=1000, batch_size=128,callbacks=[callback2], validation_data=(X_dev, y_dev))#batch_size=64 ,#validation_data=(X_dev, y_dev)

In [ ]:
model_lstm_1.evaluate(X_train, y_train)

In [ ]:
model_lstm_1.evaluate(X_dev, y_dev)

In [ ]:
model_lstm_1.save("./modelArgantine/final/v8_64_5.4")

In [ ]:
path ="./testdata/"
file_list = os.listdir(path)
file_list.sort()



In [ ]:
test1 = {}
for f in file_list:
  v=videoProc2(path+f,c=4,o=0)
  test1[f]=v
  print(f,v.shape)


In [ ]:
test2 = {}
for f in file_list: 
  v=videoProc2(path+f,c=2,o=0)
  test2[f]=v
  print(f,v.shape)

In [ ]:
test3 = {}
for f in file_list: 
  v=videoProc2(path+f,c=3,o=0)
  test3[f]=v
  print(f,v.shape)

In [ ]:
# reconstructed_model = tf.keras.models.load_model("./modelArgantine/v8.6")

In [ ]:
# reconstructed_model.summary()

In [ ]:
for l,v in test2.items():#model_lstm_1  # v  = (v-m)/s
  # v  = (v-v.mean())/(v.std()+1e-7)
  p = model_lstm_1.predict(np.reshape(v, (1,20,170)))
  pp =np.argmax(p)
  print(l,pp,p[:,pp],v[14].max())

In [ ]:
for l,v in test3.items():#model_lstm_1  # v  = (v-m)/s
  # v  = (v-m)/s
  p = model_lstm_1.predict(np.reshape(v, (1,20,170)))
  pp =np.argmax(p)
  print(l,pp,p[:,pp],v[14].max())

In [ ]:
for l,v in test1.items():#model_lstm_1  # v  = (v-m)/s
  p = model_lstm_1.predict(np.reshape(v, (1,20,170)))
  pp =np.argmax(p)
  print(l,pp,p[:,pp],v.std())